# DTox tutorial
## This notebook demonstrates in details how to implement DTox from scratch 

### 1. Import all the modules and functions 

In [1]:
# Modules
import sys
import numpy as np
import pandas as pd
import subprocess
# DTox functions
sys.path.insert(0, 'code/')
import targettox
import dtox
import dtox_interpret

### 2. Load input dataset 
DTox takes in a dataset that specifies the structural representation of each compound along with the labeled toxicity outcome (see the data frame below).

In this notebook, we adopted a subset of the mitochondria toxicity screening dataset (500 compounds for training, 500 compounds for testing) from the Tox21 program, with each compound represented by the 166-bit MACCS fingerprint. The assay outcome of each compound is binary: either active (1) or inactive (1). 

In [2]:
# load training set 
train_data_df = pd.read_csv('data/example/mitotox_example_input_train.tsv', sep = '\t', header = 0, index_col = 0)
# load testing set 
test_data_df = pd.read_csv('data/example/mitotox_example_input_test.tsv', sep = '\t', header = 0, index_col = 0)
# check the format of input data matrix
train_data_df.head()

,FP1,FP2,FP3,FP4,FP5,FP6,FP7,FP8,FP9,FP10,...,FP158,FP159,FP160,FP161,FP162,FP163,FP164,FP165,FP166,assay_outcome
CID_12938,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,0,1
CID_23581791,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
CID_338733,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,1,1,1,0,1
CID_6579,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,0
CID_45268380,0,0,0,0,0,0,0,0,0,0,...,1,0,1,1,1,1,0,1,1,0


### 3. Infer target binding profile from structural features
After loading the input dataset, users need to implement TargetTox (our previously developed method) to derive a target profile of each compound. TargetTox was pre-trained on experimentally measured compound-target binding affinities to infer the target binding probability of each compound from its MACCS fingerprint. 

The derived profile contains compound-target binding probabilities for 361 target proteins (see the data frame below)

In [3]:
# Infer target binding profile for compounds in training set 
ao_col = 'assay_outcome'
train_target_profile_df = targettox.derive_target_profile(train_data_df)
train_target_profile_df[ao_col] = train_data_df[ao_col]
# Infer target binding profile for compounds in testing set 
test_target_profile_df = targettox.derive_target_profile(test_data_df)
test_target_profile_df[ao_col] = test_data_df[ao_col]
train_target_profile_df.head()

,O00329,O00408,O00519,O14672,O14684,O14757,O14920,O14939,O15229,O15379,...,Q9UNQ0,Q9Y233,Q9Y271,Q9Y2D0,Q9Y337,Q9Y5N1,Q9Y5Y4,Q9Y5Y6,Q9Y5Z0,assay_outcome
CID_12938,0.540333,0.520000,0.559730,0.006667,0.629167,0.600000,0.404500,0.236515,0.697833,0.409167,...,0.411564,0.930000,0.824111,0.060000,0.0,0.745238,0.474167,0.175000,0.492615,1
CID_23581791,0.865000,0.940000,0.090000,0.130167,0.370000,0.980000,0.560000,0.444667,0.386000,0.360000,...,0.227826,0.577500,0.823226,0.831667,0.0,0.535238,0.710000,0.849131,0.319000,0
CID_338733,0.690000,0.350833,0.493332,0.000000,0.231500,0.463333,0.280000,0.110000,0.528357,0.137500,...,0.020000,0.870000,0.231658,0.020000,0.0,1.000000,0.570833,0.177292,0.434000,1
CID_6579,0.249167,0.080000,0.269205,0.000000,0.301333,0.130000,0.146667,0.160000,0.679221,0.370000,...,0.250000,0.764833,0.231658,0.000000,0.0,0.636667,0.418667,0.177292,0.236500,0
CID_45268380,0.950000,0.610000,0.578405,0.000000,0.369333,0.410000,0.390000,0.130000,0.274167,0.360000,...,0.177167,0.865000,0.231658,0.060000,0.0,0.641333,0.395000,0.177292,0.337333,0


### 4. Learn DTox model 

DTox has three required input arguments: (i) the input training dataset, (ii) name of column that contains label/response data ('label_col_name'), and (iii) name of folder to store output files ('out_folder').

DTox also has five hyperparameters (shown in the table below). For a detailed description of each hyperparameter, please check [the Experimental procedures section of DTox paper](https://doi.org/10.1016/j.patter.2022.100565). Users can change the default setting accroding to their own preference. 

| Hyperparameter | Annotation | Default value |
|----------------|-----------------------------------------------|----------------|
| root_process | abbreviated name of root biological processes  | GE+IS+M+ST** |
| min_pathway_size | minimal size of pathways included in DTox model | 5 |
| max_module_size | maximal size of node modules in DTox neural network model | 20 |
| auxiliary_alpha | coefficient for auxiliary loss in loss function | 0.5 |
| l2_lambda | coefficient for L2 regularization  | 0.0001 |

** GE: gene expression, IS: immune system, M: metabolism, ST: signal transduction

The function returns four items. The first item gives a summary of the DTox neural network statistics (see the dictionary 'mito_model_info' below). The second item is the learned model (stored in file 'model.pt'). The third item is the optimized loss function.  

In [4]:
result_folder = 'data/example/result/'
mito_model_info, mito_model, mito_model_loss, mito_model_training_summary = dtox.dtox(train_target_profile_df,
                                                                                      label_col_name = ao_col, 
                                                                                      out_folder = result_folder)
mito_model_info

['Number of hidden layers:12',
 'Number of input features: 361',
 'Number of hidden modules: 366,123,51,22,14,12,7,2,2,2,1,1',
 'Number of hidden neurons: 1823,949,493,257,150,149,84,21,31,37,18,20']

The fourth item summarizes the evolution of training/testing loss function over training epoches (see the data frame below), including total loss, root loss, and auxiliary loss. For a detailed description of each loss type, please check [the Experimental procedures section of DTox paper](https://doi.org/10.1016/j.patter.2022.100565).  

In [5]:
mito_model_training_summary.head()

,epoch,training_total_loss,training_root_loss,training_auxiliary_loss,testing_total_loss,testing_root_loss,testing_auxiliary_loss
0,1,4.749577,0.638738,8.221678,4.589060,0.583916,8.010288
1,2,4.430245,0.529007,7.802476,4.282406,0.484473,7.595867
2,3,4.161099,0.460645,7.400909,4.093854,0.471416,7.244876
3,4,3.988595,0.458851,7.059487,3.951758,0.470547,6.962422
4,5,3.857885,0.458740,6.798289,3.824788,0.463351,6.722874


Users can also evaluate the performance of learned DTox model using the 'dtox_eval' function. The function has five required input arguments: (i) the input training dataset, (ii) the input testing dataset, (iii) name of column that contains label/response data ('label_col_name'), (iv) the learned DTox model ('trained_model'), and (v) the loss function ('loss').

The function returns two items that summerize training and testing performance separately. Each item is a dictionary that contains the performance metrics: loss ('total_loss', 'root_loss', and 'auxiliary_loss'), AUROC ('auc'), balanced accuracy ('bac'), F1 score ('f1'), along with their confidence interval metric ('ci', shown as the half of the confidence interval width. e.g. 'auc ± auc_ci' is the confidence inverval of AUROC)

In [6]:
mito_train_perf, mito_valid_perf = dtox.dtox_eval(train_target_profile_df, test_target_profile_df, 
                                                  label_col_name = ao_col,
                                                  trained_model = mito_model, 
                                                  loss = mito_model_loss)
mito_valid_perf

{'total_loss': 3.330639362335205,
 'root_loss': 0.4365403950214386,
 'auxiliary_loss': 5.788197994232178,
 'auc': 0.7479215164615897,
 'auc_ci': 0.03747412809647982,
 'bac': 0.5659614745081988,
 'bac_ci': 0.02841727674569322,
 'f1': 0.24137931034482754,
 'f1_ci': 0.08187500000000003}

### 5. Explain DTox predictions with interpretation framework

The interpretation framework has five required input arguments: (i) data frame that contains query data instances for interpretation, (ii) the learned DTox model ('dtox_model'), (iii) the input training dataset ('dtox_combine_df'), (iv) name of column that contains label/response data ('label_col_name'), and (v) name of folder to store output files ('out_folder').

The framework also has four hyperparameters (shown in the table below). For a detailed description of each hyperparameter, please check [the Experimental procedures section of DTox paper](https://doi.org/10.1016/j.patter.2022.100565). Users can change the default setting accroding to their own preference. 

| Hyperparameter | Annotation | Default value |
|----------------|-----------------------------------------------|----------------|
| prop_rule |  propagation rule to be implemented ** | gamma-epsilon |
| rule_factor1 | first factor in LRP rules: 'gamma' or 'alpha' | 0.001 |
| rule_factor2 | second factor in LRP rules: 'epsilon or 'beta' | 0.1 |
| N_null_models | sampling times for computing empirical P-value of path relevance  | 200 |

** 'gamma-epsilon' for generic rule, 'alpha-beta' for αβ rule

The function returns two items. The first item gives the relevance scores of hidden modules (target proteins and pathways) for each data instance (stored in 'module_relevance.tsv'). 

In [7]:
# Extract all active compounds from testing set for prediction explanation 
query_interpret_df = test_target_profile_df[test_target_profile_df[ao_col] == 1]
pathway_rel_df, rel_fdr_df = dtox_interpret.dtox_interpret(query_interpret_df, 
                                                           dtox_model = mito_model, 
                                                           dtox_combine_df = train_target_profile_df, 
                                                           label_col_name = ao_col, 
                                                           out_folder = result_folder,
                                                           N_null_models = 10)
pathway_rel_df.head()

,O00329,O00408,O00519,O14672,O14684,O14757,O14920,O14939,O15229,O15379,...,R-HSA-181438,R-HSA-74160,R-HSA-111885,R-HSA-168898,R-HSA-168249,R-HSA-418594,R-HSA-168256,R-HSA-388396,R-HSA-372790,R-HSA-162582
CID_75146,0.005572,0.0,0.0,0.009452,0.000219,0.000031,0.195406,0.005284,0.002058,-0.000254,...,0.008451,-0.105616,0.001607,0.063626,0.288051,0.002643,0.515664,0.009112,-0.034647,0.005266
CID_3884,0.001451,0.0,0.0,-0.014298,-0.000046,-0.000007,0.028454,-0.001563,0.001839,0.000638,...,0.009588,0.018577,-0.000430,0.063125,0.162685,-0.001645,0.153316,-0.005901,0.002795,0.074748
CID_6733,0.000595,0.0,0.0,-0.003680,-0.000017,0.000003,0.003707,0.002112,0.000590,0.000287,...,0.002917,0.008013,-0.000128,0.019387,0.078349,-0.000433,0.075412,-0.001563,-0.000184,0.030397
CID_7572,0.000115,0.0,0.0,-0.001748,-0.000009,-0.000002,0.003190,-0.000383,0.001001,0.000022,...,0.001622,0.002456,-0.000072,0.010881,0.046482,-0.000275,0.046199,-0.000954,0.002359,0.026383
CID_7626,0.000145,0.0,0.0,-0.001957,-0.000014,0.000001,0.004087,0.001049,0.000629,0.000064,...,0.001969,0.005273,-0.000082,0.013114,0.063598,-0.000312,0.062999,-0.001101,0.002886,0.016780


The second item gives the empirical P-value and FDR of observed path relevance scores computed from permutation test.  (stored in 'path_relevance_pv.tsv'). Each path is represented by the pathways and target protein along the path, starting from the most general biological process to more specific pathways and target protein (separated by "_").  

In [8]:
rel_fdr_df.head()

,cid,path_id,observed,p_value,fdr
61,CID_10219,R-HSA-162582_R-HSA-2586552_O60674,-4.992686,0.0,0.0
61,CID_10219,R-HSA-162582_R-HSA-2028269_P42574,-4.992633,0.0,0.0
61,CID_10219,R-HSA-74160_R-HSA-73864_R-HSA-73863_P51946,-6.708735,0.0,0.0
61,CID_10219,R-HSA-168256_R-HSA-1280218_R-HSA-2132295_P07711,-7.494643,0.0,0.0
61,CID_10219,R-HSA-168256_R-HSA-1280218_R-HSA-2132295_P07858,-6.958086,0.0,0.0


Users can also visualize the interpretation results by calling the Rscript 'code/dtox_visualize.R'. The visualization function has three arguments: (i) the ID of query compound, (ii) visualization mode ('simple' for only displaying the paths identified by DTox, 'complex' for displaying all paths that involve proteins connected to the query compound), (iii) name of folder to store output files. 

The visualization plot will be stored in a '.html' file (named after the query compound) in the specified result folder (see [the plot for 'CID_2346' as an example](data/example/result/CID_2346.html)). The interactive network diagram shows the DTox structure connecting query compound (triangle node) to the toxicity outcome (rectangle node) via target proteins (diamond nodes) and pathway modules (round nodes). Pathways with relevance score > 0 are colored in purple (with the scale proportional to relevance scale), while the other irrelevant pathways are colored in white (irrelevant pathways are only shown in the 'complex' mode). The VNN paths identified for the query by DTox are represented by solid lines, while the other irrelevant paths are represented by dashed lines (irrelevant paths are only shown in the 'complex' mode).

In [9]:
# Specify plotting parameters
query_compound = 'CID_2346'
display_mode = 'simple'
# Call Rscript for visualization
plot_command = ' '.join(['Rscript', 'code/dtox_visualize.R', query_compound, display_mode, result_folder])
subprocess.call(plot_command, shell = True)

0